# 片道切符まんさんの「取引大会で優勝したときのロジック」を実素してみる。
https://note.com/_and_go/n/na62475340756

## ロジック概要

いくつかパターンがあるみたいですが、今回は以下のパターンを実装します。

1. Best rate付近の大きめの板の手前に指値を出す。
2. Buy, Sell両方にオーダーを出す。

### 考慮事項

#### 「大きめの板」の定義はどうするか？

1. 板の最初は除外。２番目の板から計算。
2. 定数　ignore_size 以下の板を無視
3. 最初の ignore_size より大きな板を壁とする。


### 執行ロジック
1. 1分に１回、もしくはオーダーが執行されたら「大きめの板」の前の価格を計算
2. 売り・買いの両方にオーダーを出す（ただし以下のようにして同じ方向のオーダーは１つにする）
   1. 売注文残がなく、かつ、買い注文が約定していてポジションがマイナスの場合 →　売り注文
   2. 買注文残がなく、かつ、売り注文が約定していてポジションがプラスの場合　→　買注文

注：現物にはポジションの概念がありませんが、Bot起動時からのセッションで売り・買いの約定数の差分をポジションとして計算しています。

## コード

BinanceのBTUCSDTのデータを使いフォーワードテストを行い、結果がグラフ表示するコードです。

以下順番に実行していってみてください。

テスト用のため、フォーワードテスト時間は１８０秒にしてありますが、適宜変更してみてください。

ignore_sizeなど変化させてみるのも面白いとおもいます。

In [ ]:
# rbot拡張ライブラリのインストール（２回目は不要です）
! pip install --upgrade pip
! pip install --upgrade rbot

In [ ]:
# 関連ライブラリのインストール
! pip install pyarrow
! pip install polars
! pip install plotly
! pip install nbformat
! pip install numpy
! pip install pandas
! pip install json2html

In [ ]:
! pip install polars
! pip install pyarrow

In [ ]:
import polars as pl
pl.Config.fmt_str_lengths = 50


In [ ]:

from rbot import BinanceConfig, BinanceMarket
from rbot import Runner
from rbot import NOW, HHMM

In [ ]:
class MMBot:
    def __init__(self):
        """ MMBot クラス初期化（チューニングパラメータ設定）"""
        self.ignore_size = 0.05
        self.order_size = 0.01
        self.price_tick = 0.01
        self.expire_time = 60*10

    def wall_price(self, board):
        """板の壁の値段を返す"""
        wall = board[1:].filter(self.ignore_size < pl.col("size"))
        if len(wall) == 0:
            return None
        
        price = wall.head(1)['price'][0]
        return price


    def main_logic(self, session):
        """ メインロジック """
        if session.expire_order(self.expire_time):  # １０分以上経過した注文をキャンセル
            return                                  # キャンセルしたら終了(次のループで再度注文する)
        
        bid, ask = session.board # 板情報を取得
        if len(bid) < 10 or len(ask) <10:  # 板情報がない場合は終了
            return 
        
        buy_price = self.wall_price(bid)    # 壁が検出できた場合
        if buy_price is None:
            return
        buy_price = buy_price + self.price_tick    # 壁の一つ前の価格を計算
        
        sell_price = self.wall_price(ask)
        if sell_price is None:
            return
        sell_price = sell_price - self.price_tick       # 壁の一つ前の価格を計算
        
        session.log_indicator("buy_price", buy_price)         # ログに壁の一つ前の価格を記録   
        session.log_indicator("sell_price", sell_price)       # ログに壁の一つ前の価格を記録
        session.log_indicator("spread", sell_price - buy_price)   # 買いと売りの壁の差を記録
        
        if not session.buy_orders and session.position <= 0:    # 買い注文がなく、ポジションがマイナス（売り注文が約定済みの場合）
            session.limit_order("Buy", buy_price, self.order_size)  # 壁の一つ前の価格で買い注文を出す
            print("Buy order price", buy_price, "size", self.order_size)
        
        if not session.sell_orders and 0 <= session.position:      # 売り注文がなく、ポジションがプラス（買い注文が約定済みの場合）
            session.limit_order("Sell", sell_price, self.order_size) # 壁の一つ前の価格で売り注文を出す
            print("Sell order price", sell_price, "size", self.order_size)


    def on_init(self, session):
        """ フレームワークから呼び出される初期化 on_clockの呼び出し間隔を設定 """
        session.clock_interval_sec = 60     # 60秒ごとに on_clock を呼び出す

    def on_clock(self, session, timestamp):
        """ フレームワークから呼び出される定期的な処理 """
        self.main_logic(session)
    
    def on_update(self, session, order):
        """ フレームワークから呼び出される注文更新時の処理 """
        # 注文が約定したかキャンセルされたら、次のオーダーを出すためにメインロジックを呼び出す
        if order.status == "Filled" or order.status == "Canceled":
            self.main_logic(session)


In [ ]:
config = BinanceConfig.TEST_BTCUSDT
market = BinanceMarket(config)

from rbot import init_log
init_log()


In [ ]:
market.download(1, verbose=True)

In [ ]:
agent = MMBot()         # テスト対象のエージェントのインスタンスを作成
runner = Runner()       # テスト実行クラス（Runner）のインスタンスを作成

# ドライラン（フォーワードテスト実施）
# 最低１日分のログをダウンロードするため、初回は時間がかかります。
session = runner.real_run(
                        market=market,            # テスト対象のマーケット()
                        agent=agent,             # テスト対象のエージェント
                        verbose=True,      # 実行ログを出力する(True)
                        execute_time=180,   # テスト実行時間（秒）本番時はどは0にして無制限にする
                        log_memory=True    # ログをメモリに出力する(True)。本番時はFalseにし、ファイルに出力する。
)


# runnner.back_test()を使うと、過去のデータを使ってバックテストができるが、板情報は未対応のため今回は利用不可。
# なお、runner.dry_run()をrunner.run()に変えると本当に注文が出されるので注意！！

In [ ]:
log = session.log       # ログオブジェクトを取得

orders = log.orders    # ログオブジェクトから注文ログを取得
orders                  # 注文ログを表示

In [ ]:
# 損益結果
orders['sum_profit'][-1]

In [ ]:
# 利益ピーク
orders['sum_profit'].max()

In [ ]:
# 利益最小
orders['sum_profit'].min()

In [ ]:
# １回の取引の最大利益
orders['total_profit'].max()

In [ ]:
# １回の取引の最大損失
orders['total_profit'].min()

In [ ]:
# オーダー状況分析
orders.group_by(['order_side', 'status']).count()

In [ ]:
# オーダーごとに集約
group_by_order = orders.group_by(['order_id']).agg(
    pl.col('symbol').first(), 
    pl.col('order_side').first(), 
    pl.col('status').last(), 
    pl.col('order_price').first(), 
    pl.col('order_size').first(), 
    pl.col('execute_size').sum(),
    pl.col('update_time').last(),
    pl.col('total_profit').sum()
).sort('update_time')


group_by_order

In [ ]:
# 勝敗分析
lost = len(group_by_order.filter(pl.col('total_profit') < 0))
win = len(group_by_order.filter(pl.col('total_profit') > 0))

print(f'勝ち:{win} 負け:{lost} 勝率:{win/(win+lost)}')

In [ ]:
# グラフで状況確認

In [ ]:

ohlcv = market.ohlcv(runner.start_timestamp - HHMM(0,1),  # テスト時間の１分前から
                     runner.last_timestamp + HHMM(0, 1),  # テスト時間の１分後まで
                      5                    # 5秒足)
                      )

ohlcv

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.1, 0.1, 0.1, 0.6],
                    subplot_titles=("spread", "position", "profit", "candlestick"))

# row 1 (indicator)
spread = log['spread']
fig.add_trace(go.Scatter(x=spread['timestamp'], y=spread['spread'], name="spread"), row=1, col=1)

# row 2 (position)
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['position'], name="position", line=dict(shape='hv')), row=2, col=1)

# row 3 (profit)
profit = orders['sum_profit']
fig.add_trace(go.Scatter(x=orders['update_time'], y=orders['sum_profit'], name="profit", line=dict(shape='hv')), row=3, col=1)

# row 4 (candlestick)
fig.add_candlestick(x=ohlcv['timestamp'], open=ohlcv['open'], high=ohlcv['high'], low=ohlcv['low'], close=ohlcv['close'], row=4, col=1)

# row 4 (order)
buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='arrow-up', color='red', size=10), name="buy"), row=4, col=1)

buy_orders = orders.filter((orders['order_side'] == 'Buy') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=buy_orders['update_time'], y=buy_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='red', size=10), name="buy filled"), row=4, col=1)


sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'New'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='arrow-down', color='blue', size=10), name="sell"), row=4, col=1)

sell_orders = orders.filter((orders['order_side'] == 'Sell') & (orders['status'] == 'Filled'))
fig.add_trace(go.Scatter(x=sell_orders['update_time'], y=sell_orders['order_price'], mode='markers', marker=dict(symbol='cross-thin-open', color='blue', size=10), name="sell filled"), row=4, col=1)

fig.update_layout(height=800, title_text="Backtest Result")

# （おまけ）該当期間のVAP(Volume At Price)の表示

In [ ]:
vap = market.vap(start_time=runner.start_timestamp, 
                 end_time=runner.last_timestamp,
                 price_unit=1
                 )

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=vap['sell_volume'],
            y=vap['price'],
            fill='tozerox',
            name='sell'
        ),
        go.Scatter(
            x=vap['sell_volume'] + vap['buy_volume'],
            y=vap['price'],
            fill='tonextx',
            name='buy'
        ),
    ],
    layout=go.Layout(barmode='stack')
)

fig